## 学習用データセットのjsonファイルを作成する(test,validation,trainに分割する)

In [3]:
import os
import json
import random

# -------------------------
# ハードコーディング設定
# -------------------------
INPUT_JSON = "./../json/eat_not_aug.json"        # マスター JSON ファイルのパス（実際のパスに変更してください）
OUTPUT_DIR = "./../json/ssl_gru_only_eat_not_aug"               # 分割後の JSON ファイルの出力先ディレクトリ
TRAIN_VAL_SPLIT = 0.9                        # 残りのデータに対する train 側の割合（例: 0.8 → 80% が train, 20% が val）
TEST_SPEAKERS = ["MAN01", "MDK01", "MDN01", "MKG01", "MHF01"]            # テストに含める speaker 名のリスト
# ※ アノテーション内の "path" からファイル名を取得し、"_" で区切ったときの第2要素を speaker 名とみなして判断します。

# -------------------------
# 出力先ディレクトリの作成
# -------------------------
os.makedirs(OUTPUT_DIR, exist_ok=True)

# -------------------------
# マスター JSON の読み込み
# -------------------------
with open(INPUT_JSON, 'r', encoding='utf-8') as f:
    annotations = json.load(f)

# -------------------------
# アノテーションをテスト対象と残りに分割
# -------------------------
test_annotations = []
remain_annotations = []

for ann in annotations:
    wav_path = ann.get("path", "")
    filename = os.path.basename(wav_path)
    parts = filename.split("_")
    if len(parts) >= 2 and parts[1] in TEST_SPEAKERS:
        test_annotations.append(ann)
    else:
        remain_annotations.append(ann)

# -------------------------
# 残りのデータをシャッフルして train と val に分割
# -------------------------
random.shuffle(remain_annotations)
split_idx = int(len(remain_annotations) * TRAIN_VAL_SPLIT)
train_annotations = remain_annotations[:split_idx]
val_annotations = remain_annotations[split_idx:]

# -------------------------
# 分割した各データセットを JSON として保存
# -------------------------
train_json_path = os.path.join(OUTPUT_DIR, "train.json")
val_json_path = os.path.join(OUTPUT_DIR, "val.json")
test_json_path = os.path.join(OUTPUT_DIR, "test.json")

with open(train_json_path, 'w', encoding='utf-8') as f:
    json.dump(train_annotations, f, ensure_ascii=False, indent=2)
with open(val_json_path, 'w', encoding='utf-8') as f:
    json.dump(val_annotations, f, ensure_ascii=False, indent=2)
with open(test_json_path, 'w', encoding='utf-8') as f:
    json.dump(test_annotations, f, ensure_ascii=False, indent=2)

# -------------------------
# 結果の表示
# -------------------------
print("データセットの分割が完了しました。")
print(f"Train: {len(train_annotations)} 件")
print(f"Val  : {len(val_annotations)} 件")
print(f"Test : {len(test_annotations)} 件")
print(f"出力先: {OUTPUT_DIR}")


データセットの分割が完了しました。
Train: 1755 件
Val  : 195 件
Test : 400 件
出力先: ./../json/ssl_gru_only_eat_not_aug
